In [3]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io, measure
from skimage.morphology import thin, skeletonize
from skimage.measure import regionprops
from scipy import ndimage
from scipy.ndimage import label as scipy_label
from skimage.measure import label as skimage_label
import tifffile
import seaborn as sns
from pathlib import Path

# Set up matplotlib for better visualization
plt.rcParams['figure.figsize'] = (12, 10)
plt.rcParams['figure.dpi'] = 100
plt.style.use('ggplot')

# Define your input and output paths
input_dir = "/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/flow3-x20/Membrane"
output_dir = "/content/drive/MyDrive/knowledge/University/Master/Thesis/Analysis/flow3-x20/Overall/Membrane"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Function to extract pressure value from filenames
def extract_pressure(filename):
    """Extract pressure value from the filename using regex."""
    # Look for patterns like '1.4Pa' or '0Pa'
    pressure_match = re.search(r'(\d+\.?\d*)Pa', filename)
    if pressure_match:
        try:
            return float(pressure_match.group(1))
        except ValueError:
            print(f"Warning: Could not convert pressure value in {filename} to float.")
            return None

    # If no match found with the Pa pattern, try looking for specific pressure indicators
    if '0Pa' in filename or '_0Pa_' in filename:
        return 0.0
    elif '1.4Pa' in filename or '_1.4Pa_' in filename:
        return 1.4

    print(f"Warning: Could not extract pressure value from {filename}")
    return None

# Function to categorize files by pressure
def categorize_files_by_pressure(input_dir):
    """Sort files based on pressure values."""
    pressure_files = {}

    # Get all TIFF files
    files = [f for f in os.listdir(input_dir) if f.endswith(('.tif', '.tiff'))]
    print(f"Found {len(files)} TIFF files in the input directory.")

    for file in files:
        pressure = extract_pressure(file)
        if pressure is not None:
            if pressure not in pressure_files:
                pressure_files[pressure] = []
            pressure_files[pressure].append(os.path.join(input_dir, file))

    print(f"Found {len(pressure_files)} different pressure values: {list(pressure_files.keys())}")
    return pressure_files

# Function to calculate membrane tortuosity
def calculate_tortuosity(membrane_image, min_length=10):
    """
    Calculate tortuosity for membrane segments.

    Parameters:
    membrane_image: Binary image of membrane
    min_length: Minimum length to consider (to avoid very short segments)

    Returns:
    Dictionary with tortuosity values and statistics
    """
    # Ensure binary
    binary_membrane = membrane_image > 0

    # Create skeleton
    skeleton = thin(binary_membrane)

    # Label the skeletonized segments
    labeled_skeleton, num_segments = scipy_label(skeleton)
    print(f"Number of separate membrane segments: {num_segments}")

    # Calculate properties for each segment
    props = regionprops(labeled_skeleton)

    # Calculate tortuosity for each segment
    tortuosity_values = []
    segment_lengths = []
    segment_distances = []

    for prop in props:
        # Get actual path length (number of pixels in the segment)
        actual_length = prop.area  # for skeleton, area equals length

        # Get endpoint distance (Euclidean distance between furthest points)
        coords = prop.coords
        if len(coords) > min_length:  # only consider segments longer than min_length
            distances = np.sqrt(np.sum((coords[:, None, :] - coords[None, :, :]) ** 2, axis=2))
            endpoint_distance = np.max(distances)

            segment_lengths.append(actual_length)
            segment_distances.append(endpoint_distance)

            # Calculate tortuosity (ratio of actual length to endpoint distance)
            if endpoint_distance > 0:
                tortuosity = actual_length / endpoint_distance
                tortuosity_values.append(tortuosity)

    # Calculate statistics
    stats = {
        'num_segments': len(tortuosity_values),
        'mean_tortuosity': np.mean(tortuosity_values) if tortuosity_values else 0,
        'median_tortuosity': np.median(tortuosity_values) if tortuosity_values else 0,
        'min_tortuosity': np.min(tortuosity_values) if tortuosity_values else 0,
        'max_tortuosity': np.max(tortuosity_values) if tortuosity_values else 0,
        'std_tortuosity': np.std(tortuosity_values) if tortuosity_values else 0,
        'tortuosity_values': tortuosity_values,
        'segment_lengths': segment_lengths,
        'segment_distances': segment_distances,
        'skeleton': skeleton
    }

    return stats

# Function to calculate membrane coverage
def calculate_coverage(membrane_image, total_area=None):
    """
    Calculate membrane coverage as a percentage of total area.

    Parameters:
    membrane_image: Binary image of membrane
    total_area: Total area to calculate percentage (if None, uses image dimensions)

    Returns:
    Dictionary with coverage metrics
    """
    # Ensure binary
    binary_membrane = membrane_image > 0

    # Calculate membrane area
    membrane_area = np.sum(binary_membrane)

    # If total_area is not provided, use the entire image dimensions
    if total_area is None:
        total_area = binary_membrane.shape[0] * binary_membrane.shape[1]

    # Calculate coverage percentage
    coverage_percentage = (membrane_area / total_area) * 100

    # Calculate density (membrane pixels per 1000 pixels)
    density = (membrane_area / total_area) * 1000

    stats = {
        'membrane_area': membrane_area,
        'total_area': total_area,
        'coverage_percentage': coverage_percentage,
        'density': density
    }

    return stats

# Function to calculate membrane complexity metrics
def calculate_complexity(membrane_image):
    """
    Calculate complexity metrics for membrane.

    Parameters:
    membrane_image: Binary image of membrane

    Returns:
    Dictionary with complexity metrics
    """
    # Ensure binary
    binary_membrane = membrane_image > 0

    # Skeletonize
    skeleton = skeletonize(binary_membrane)

    # Find branch points and endpoints
    branch_points = np.zeros_like(skeleton)
    endpoints = np.zeros_like(skeleton)

    # Use convolution to find branch points
    kernel = np.ones((3, 3))
    kernel[1, 1] = 0  # Center pixel
    neighbors = ndimage.convolve(skeleton.astype(int), kernel, mode='constant', cval=0)

    # Branch points have 3 or more neighbors
    branch_points[np.logical_and(skeleton, neighbors >= 3)] = 1

    # Endpoints have exactly 1 neighbor
    endpoints[np.logical_and(skeleton, neighbors == 1)] = 1

    # Count branch points and endpoints
    num_branch_points = np.sum(branch_points)
    num_endpoints = np.sum(endpoints)

    # Calculate segment length
    if num_endpoints > 0:
        avg_segment_length = np.sum(skeleton) / num_endpoints
    else:
        avg_segment_length = 0

    # Calculate branching complexity (branch points / total skeleton points)
    if np.sum(skeleton) > 0:
        branching_complexity = num_branch_points / np.sum(skeleton)
    else:
        branching_complexity = 0

    stats = {
        'num_branch_points': num_branch_points,
        'num_endpoints': num_endpoints,
        'avg_segment_length': avg_segment_length,
        'branching_complexity': branching_complexity,
        'skeleton': skeleton,
        'branch_points': branch_points,
        'endpoints': endpoints
    }

    return stats

# Function to visualize membrane metrics
def visualize_membrane_metrics(membrane_image, tortuosity_stats, coverage_stats, complexity_stats, filename, output_path):
    """Create comprehensive visualization of membrane metrics"""
    fig, axes = plt.subplots(3, 2, figsize=(16, 18))

    # Original image and skeleton
    axes[0, 0].imshow(membrane_image, cmap='gray')
    axes[0, 0].set_title("Original Membrane Image")
    axes[0, 0].axis('off')

    axes[0, 1].imshow(tortuosity_stats['skeleton'], cmap='gray')
    axes[0, 1].set_title("Membrane Skeleton")
    axes[0, 1].axis('off')

    # Tortuosity histogram
    if tortuosity_stats['tortuosity_values']:
        sns.histplot(tortuosity_stats['tortuosity_values'], bins=20, kde=True, ax=axes[1, 0])
        axes[1, 0].set_title(f"Tortuosity Distribution (Mean: {tortuosity_stats['mean_tortuosity']:.3f})")
        axes[1, 0].set_xlabel("Tortuosity")
        axes[1, 0].set_ylabel("Frequency")
    else:
        axes[1, 0].text(0.5, 0.5, "No tortuosity data available",
                      horizontalalignment='center', verticalalignment='center')

    # Tortuosity scatter plot
    if tortuosity_stats['segment_lengths'] and tortuosity_stats['segment_distances']:
        axes[1, 1].scatter(tortuosity_stats['segment_distances'], tortuosity_stats['segment_lengths'], alpha=0.5)
        axes[1, 1].plot([0, max(tortuosity_stats['segment_distances'])],
                       [0, max(tortuosity_stats['segment_distances'])],
                       'r--', label='Tortuosity = 1')
        axes[1, 1].set_title("Segment Length vs Endpoint Distance")
        axes[1, 1].set_xlabel("Endpoint Distance (pixels)")
        axes[1, 1].set_ylabel("Segment Length (pixels)")
        axes[1, 1].legend()
    else:
        axes[1, 1].text(0.5, 0.5, "No segment data available",
                      horizontalalignment='center', verticalalignment='center')

    # Complexity visualization
    branch_image = np.zeros((*membrane_image.shape, 3), dtype=np.uint8)

    # Skeleton in green
    branch_image[complexity_stats['skeleton'] > 0] = [0, 255, 0]

    # Branch points in red
    branch_image[complexity_stats['branch_points'] > 0] = [255, 0, 0]

    # Endpoints in blue
    branch_image[complexity_stats['endpoints'] > 0] = [0, 0, 255]

    axes[2, 0].imshow(branch_image)
    axes[2, 0].set_title(f"Skeleton Structure (BP: {complexity_stats['num_branch_points']}, EP: {complexity_stats['num_endpoints']})")
    axes[2, 0].axis('off')

    # Coverage visualization - create a bar chart
    coverage_data = {
        'Metric': ['Coverage (%)', 'Membrane Density (per 1000px)'],
        'Value': [coverage_stats['coverage_percentage'], coverage_stats['density']]
    }

    sns.barplot(x='Metric', y='Value', data=pd.DataFrame(coverage_data), ax=axes[2, 1])
    axes[2, 1].set_title("Membrane Coverage")
    axes[2, 1].set_ylim(0, max(30, coverage_stats['coverage_percentage'] * 1.2))  # Adjust ylim for better visualization

    # Add file info at the top of the figure
    plt.suptitle(f"Membrane Analysis - {os.path.basename(filename)}", fontsize=16, y=0.98)

    plt.tight_layout()
    plt.subplots_adjust(top=0.95)

    # Save figure
    plt.savefig(output_path, dpi=300)
    print(f"Saved visualization to {output_path}")

    plt.close()

# Function to process a single membrane image
def process_membrane_image(filepath, output_dir):
    """Process a single membrane image and calculate metrics"""
    print(f"\n=== Processing membrane image: {os.path.basename(filepath)} ===")

    try:
        # Load image
        membrane_image = io.imread(filepath)
        print(f"Loaded membrane image: shape {membrane_image.shape}")

        # Ensure image is binary (if not already)
        if membrane_image.dtype != bool:
            # If grayscale, threshold it (adjust threshold as needed)
            if len(membrane_image.shape) == 2:
                threshold = 0
                binary_membrane = membrane_image > threshold
            else:
                print(f"Warning: Non-binary image detected with shape {membrane_image.shape}")
                # For RGB images, convert to grayscale first
                if len(membrane_image.shape) == 3 and membrane_image.shape[2] == 3:
                    from skimage.color import rgb2gray
                    grayscale = rgb2gray(membrane_image)
                    binary_membrane = grayscale > 0.5
                else:
                    binary_membrane = membrane_image > 0

            # Use the binary version
            membrane_image = binary_membrane

        # Get base filename without extension for naming output files
        base_filename = os.path.splitext(os.path.basename(filepath))[0]

        # Calculate metrics
        tortuosity_stats = calculate_tortuosity(membrane_image)
        coverage_stats = calculate_coverage(membrane_image)
        complexity_stats = calculate_complexity(membrane_image)

        # Print statistics
        print(f"\nTortuosity Analysis:")
        print(f"Number of analyzed segments: {tortuosity_stats['num_segments']}")
        print(f"Mean tortuosity: {tortuosity_stats['mean_tortuosity']:.3f}")

        print(f"\nCoverage Analysis:")
        print(f"Membrane area: {coverage_stats['membrane_area']} pixels")
        print(f"Coverage percentage: {coverage_stats['coverage_percentage']:.2f}%")
        print(f"Membrane density: {coverage_stats['density']:.2f} per 1000 pixels")

        print(f"\nComplexity Analysis:")
        print(f"Number of branch points: {complexity_stats['num_branch_points']}")
        print(f"Number of endpoints: {complexity_stats['num_endpoints']}")
        print(f"Average segment length: {complexity_stats['avg_segment_length']:.2f} pixels")
        print(f"Branching complexity: {complexity_stats['branching_complexity']:.4f}")

        # Save results
        results = {
            'filename': base_filename,
            'pressure': extract_pressure(base_filename),

            # Tortuosity metrics
            'num_segments': tortuosity_stats['num_segments'],
            'mean_tortuosity': tortuosity_stats['mean_tortuosity'],
            'median_tortuosity': tortuosity_stats['median_tortuosity'],
            'min_tortuosity': tortuosity_stats['min_tortuosity'],
            'max_tortuosity': tortuosity_stats['max_tortuosity'],
            'std_tortuosity': tortuosity_stats['std_tortuosity'],

            # Coverage metrics
            'membrane_area': coverage_stats['membrane_area'],
            'total_area': coverage_stats['total_area'],
            'coverage_percentage': coverage_stats['coverage_percentage'],
            'density': coverage_stats['density'],

            # Complexity metrics
            'num_branch_points': complexity_stats['num_branch_points'],
            'num_endpoints': complexity_stats['num_endpoints'],
            'avg_segment_length': complexity_stats['avg_segment_length'],
            'branching_complexity': complexity_stats['branching_complexity']
        }

        # Save metrics as CSV (individual file)
        results_df = pd.DataFrame([results])
        results_output = os.path.join(output_dir, f"{base_filename}_metrics.csv")
        results_df.to_csv(results_output, index=False)
        print(f"Saved metrics to {results_output}")

        # Save visualization
        vis_output = os.path.join(output_dir, f"{base_filename}_analysis.png")
        visualize_membrane_metrics(membrane_image, tortuosity_stats, coverage_stats,
                                  complexity_stats, filepath, vis_output)

        return results

    except Exception as e:
        print(f"Error processing image {filepath}: {str(e)}")
        return None

    # Create filename-based output directory
    base_filename = os.path.splitext(os.path.basename(filepath))[0]
    image_output_dir = os.path.join(output_dir, base_filename)
    os.makedirs(image_output_dir, exist_ok=True)

    # Calculate metrics
    tortuosity_stats = calculate_tortuosity(membrane_image)
    coverage_stats = calculate_coverage(membrane_image)
    complexity_stats = calculate_complexity(membrane_image)

    # Print statistics
    print(f"\nTortuosity Analysis:")
    print(f"Number of analyzed segments: {tortuosity_stats['num_segments']}")
    print(f"Mean tortuosity: {tortuosity_stats['mean_tortuosity']:.3f}")

    print(f"\nCoverage Analysis:")
    print(f"Membrane area: {coverage_stats['membrane_area']} pixels")
    print(f"Coverage percentage: {coverage_stats['coverage_percentage']:.2f}%")
    print(f"Membrane density: {coverage_stats['density']:.2f} per 1000 pixels")

    print(f"\nComplexity Analysis:")
    print(f"Number of branch points: {complexity_stats['num_branch_points']}")
    print(f"Number of endpoints: {complexity_stats['num_endpoints']}")
    print(f"Average segment length: {complexity_stats['avg_segment_length']:.2f} pixels")
    print(f"Branching complexity: {complexity_stats['branching_complexity']:.4f}")

    # Save results
    results = {
        'filename': base_filename,
        'pressure': extract_pressure(base_filename),

        # Tortuosity metrics
        'num_segments': tortuosity_stats['num_segments'],
        'mean_tortuosity': tortuosity_stats['mean_tortuosity'],
        'median_tortuosity': tortuosity_stats['median_tortuosity'],
        'min_tortuosity': tortuosity_stats['min_tortuosity'],
        'max_tortuosity': tortuosity_stats['max_tortuosity'],
        'std_tortuosity': tortuosity_stats['std_tortuosity'],

        # Coverage metrics
        'membrane_area': coverage_stats['membrane_area'],
        'total_area': coverage_stats['total_area'],
        'coverage_percentage': coverage_stats['coverage_percentage'],
        'density': coverage_stats['density'],

        # Complexity metrics
        'num_branch_points': complexity_stats['num_branch_points'],
        'num_endpoints': complexity_stats['num_endpoints'],
        'avg_segment_length': complexity_stats['avg_segment_length'],
        'branching_complexity': complexity_stats['branching_complexity']
    }

    # Save metrics as CSV
    results_df = pd.DataFrame([results])
    results_output = os.path.join(image_output_dir, "membrane_metrics.csv")
    results_df.to_csv(results_output, index=False)
    print(f"Saved metrics to {results_output}")

    # Save visualization
    vis_output = os.path.join(image_output_dir, "membrane_analysis.png")
    visualize_membrane_metrics(membrane_image, tortuosity_stats, coverage_stats,
                              complexity_stats, filepath, vis_output)

    return results

# Function to analyze membranes by pressure
def analyze_membranes_by_pressure(input_dir, output_dir):
    """Analyze membrane metrics grouped by pressure values"""

    # Categorize files by pressure
    pressure_files = categorize_files_by_pressure(input_dir)

    if not pressure_files:
        print("No files with pressure information found. Exiting.")
        return

    # Process each image and collect results
    all_results = []

    for pressure, files in pressure_files.items():
        print(f"\n=== Processing {len(files)} files for pressure {pressure}Pa ===")

        # Process each file for this pressure
        pressure_results = []
        for filepath in files:
            results = process_membrane_image(filepath, output_dir)
            if results:
                pressure_results.append(results)
                all_results.append(results)

        # Combine results for this pressure
        if pressure_results:
            pressure_df = pd.DataFrame(pressure_results)
            pressure_output = os.path.join(output_dir, f"combined_metrics_{pressure}Pa.csv")
            pressure_df.to_csv(pressure_output, index=False)
            print(f"Saved combined metrics for {pressure}Pa to {pressure_output}")

    # Create combined results for all pressures
    if all_results:
        all_df = pd.DataFrame(all_results)
        all_output = os.path.join(output_dir, "all_membrane_metrics.csv")
        all_df.to_csv(all_output, index=False)
        print(f"Saved all metrics to {all_output}")

        # Create comparative visualizations across pressures
        create_pressure_comparisons(all_df, output_dir)

# Function to create comparative visualizations across pressure values
def create_pressure_comparisons(results_df, output_dir):
    """Create visualizations comparing metrics across pressure values"""

    # Make sure we have pressure values
    if 'pressure' not in results_df.columns or results_df.empty:
        print("No pressure data available for comparison")
        return

    # Group by pressure
    grouped = results_df.groupby('pressure')

    # 1. Tortuosity by pressure
    plt.figure(figsize=(12, 8))

    # Boxplot for tortuosity distribution by pressure
    sns.boxplot(x='pressure', y='mean_tortuosity', data=results_df)
    plt.title("Membrane Tortuosity by Pressure")
    plt.xlabel("Pressure (Pa)")
    plt.ylabel("Mean Tortuosity")
    plt.grid(True, alpha=0.3)

    tortuosity_output = os.path.join(output_dir, "tortuosity_by_pressure.png")
    plt.savefig(tortuosity_output, dpi=300)
    plt.close()

    # 2. Coverage by pressure
    plt.figure(figsize=(12, 8))

    sns.boxplot(x='pressure', y='coverage_percentage', data=results_df)
    plt.title("Membrane Coverage by Pressure")
    plt.xlabel("Pressure (Pa)")
    plt.ylabel("Coverage (%)")
    plt.grid(True, alpha=0.3)

    coverage_output = os.path.join(output_dir, "coverage_by_pressure.png")
    plt.savefig(coverage_output, dpi=300)
    plt.close()

    # 3. Complexity by pressure
    plt.figure(figsize=(12, 8))

    sns.boxplot(x='pressure', y='branching_complexity', data=results_df)
    plt.title("Membrane Branching Complexity by Pressure")
    plt.xlabel("Pressure (Pa)")
    plt.ylabel("Branching Complexity")
    plt.grid(True, alpha=0.3)

    complexity_output = os.path.join(output_dir, "complexity_by_pressure.png")
    plt.savefig(complexity_output, dpi=300)
    plt.close()

    # 4. Summary statistics by pressure
    summary = grouped.agg({
        'mean_tortuosity': ['mean', 'std', 'min', 'max', 'count'],
        'coverage_percentage': ['mean', 'std', 'min', 'max'],
        'branching_complexity': ['mean', 'std', 'min', 'max'],
        'num_branch_points': ['mean'],
        'avg_segment_length': ['mean']
    }).reset_index()

    # Save summary statistics
    summary_output = os.path.join(output_dir, "pressure_comparison_summary.csv")
    summary.to_csv(summary_output)
    print(f"Saved pressure comparison summary to {summary_output}")

    # 5. Multi-metric comparison
    plt.figure(figsize=(15, 10))

    # Calculate mean values for each metric by pressure
    metric_means = results_df.groupby('pressure').agg({
        'mean_tortuosity': 'mean',
        'coverage_percentage': 'mean',
        'branching_complexity': 'mean',
        'density': 'mean'
    }).reset_index()

    # Create subplots
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))

    # Plot each metric
    sns.barplot(x='pressure', y='mean_tortuosity', data=metric_means, ax=axes[0, 0])
    axes[0, 0].set_title("Mean Tortuosity")
    axes[0, 0].set_xlabel("Pressure (Pa)")
    axes[0, 0].set_ylabel("Tortuosity")

    sns.barplot(x='pressure', y='coverage_percentage', data=metric_means, ax=axes[0, 1])
    axes[0, 1].set_title("Coverage Percentage")
    axes[0, 1].set_xlabel("Pressure (Pa)")
    axes[0, 1].set_ylabel("Coverage (%)")

    sns.barplot(x='pressure', y='branching_complexity', data=metric_means, ax=axes[1, 0])
    axes[1, 0].set_title("Branching Complexity")
    axes[1, 0].set_xlabel("Pressure (Pa)")
    axes[1, 0].set_ylabel("Complexity")

    sns.barplot(x='pressure', y='density', data=metric_means, ax=axes[1, 1])
    axes[1, 1].set_title("Membrane Density")
    axes[1, 1].set_xlabel("Pressure (Pa)")
    axes[1, 1].set_ylabel("Density (per 1000px)")

    plt.suptitle("Membrane Metrics Comparison by Pressure", fontsize=16, y=0.98)
    plt.tight_layout()
    plt.subplots_adjust(top=0.93)

    multi_output = os.path.join(output_dir, "all_metrics_by_pressure.png")
    plt.savefig(multi_output, dpi=300)
    plt.close()

    print(f"Saved pressure comparison visualizations to {output_dir}")

    # Statistical significance testing between pressure groups
    if len(grouped.groups) > 1:
        perform_statistical_analysis(results_df, output_dir)

# Function to perform statistical analysis between pressure groups
def perform_statistical_analysis(results_df, output_dir):
    """Perform statistical tests to compare metrics between pressure groups"""
    try:
        from scipy import stats

        # Create output directory for statistical analysis
        stats_dir = os.path.join(output_dir, "Statistical_Analysis")
        os.makedirs(stats_dir, exist_ok=True)

        # Get unique pressure values
        pressures = sorted(results_df['pressure'].unique())

        # Metrics to compare
        metrics = ['mean_tortuosity', 'coverage_percentage', 'branching_complexity', 'density']

        # Store statistical results
        statistical_results = []

        # Perform pairwise comparisons for each metric
        for metric in metrics:
            for i in range(len(pressures)):
                for j in range(i+1, len(pressures)):
                    pressure1 = pressures[i]
                    pressure2 = pressures[j]

                    group1 = results_df[results_df['pressure'] == pressure1][metric]
                    group2 = results_df[results_df['pressure'] == pressure2][metric]

                    if len(group1) > 1 and len(group2) > 1:
                        # Perform t-test
                        t_stat, p_value = stats.ttest_ind(group1, group2, equal_var=False)

                        # Store result
                        result = {
                            'metric': metric,
                            'pressure1': pressure1,
                            'pressure2': pressure2,
                            'mean1': group1.mean(),
                            'mean2': group2.mean(),
                            'std1': group1.std(),
                            'std2': group2.std(),
                            't_statistic': t_stat,
                            'p_value': p_value,
                            'significant': p_value < 0.05
                        }

                        statistical_results.append(result)

        # Save statistical results
        if statistical_results:
            stats_df = pd.DataFrame(statistical_results)
            stats_output = os.path.join(stats_dir, "pressure_statistical_tests.csv")
            stats_df.to_csv(stats_output, index=False)
            print(f"Saved statistical analysis to {stats_output}")

            # Create visualization of significant differences
            plt.figure(figsize=(14, 10))

            # Filter for significant results
            sig_results = stats_df[stats_df['significant']]

            if not sig_results.empty:
                # Create bar chart of significant differences
                sig_results['comparison'] = sig_results.apply(
                    lambda x: f"{x['metric']}: {x['pressure1']}Pa vs {x['pressure2']}Pa", axis=1)
                sig_results['difference'] = sig_results['mean2'] - sig_results['mean1']

                # Sort by metric and absolute difference
                sig_results['abs_diff'] = abs(sig_results['difference'])
                sig_results = sig_results.sort_values(['metric', 'abs_diff'], ascending=[True, False])

                # Plot
                colors = ['green' if x > 0 else 'red' for x in sig_results['difference']]
                plt.barh(sig_results['comparison'], sig_results['difference'], color=colors)
                plt.axvline(x=0, color='black', linestyle='-', alpha=0.3)
                plt.title("Significant Differences Between Pressure Groups")
                plt.xlabel("Difference (Higher Pressure - Lower Pressure)")
                plt.grid(True, alpha=0.3)

                sig_output = os.path.join(stats_dir, "significant_differences.png")
                plt.savefig(sig_output, dpi=300)
                plt.close()
                print(f"Saved visualization of significant differences to {sig_output}")
            else:
                print("No significant differences found between pressure groups.")

    except ImportError:
        print("SciPy not available for statistical analysis. Skipping this step.")

# Main function to run the analysis
def main():
    print(f"Starting membrane analysis by pressure...")
    print(f"Input directory: {input_dir}")
    print(f"Output directory: {output_dir}")

    # Run the analysis
    analyze_membranes_by_pressure(input_dir, output_dir)

    print(f"Analysis complete. Results saved to {output_dir}")

# Run the main function
if __name__ == "__main__":
    main()

Starting membrane analysis by pressure...
Input directory: /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/flow3-x20/Membrane
Output directory: /content/drive/MyDrive/knowledge/University/Master/Thesis/Analysis/flow3-x20/Overall/Membrane
Found 8 TIFF files in the input directory.
Found 2 different pressure values: [0.0, 1.4]

=== Processing 3 files for pressure 0.0Pa ===

=== Processing membrane image: denoised_0Pa_U_05mar19_20x_L2RA_Flat_seq001_Nuclei_regional_tophat_cadherins_mask_cleaned.tif ===
Loaded membrane image: shape (1024, 1024)
Number of separate membrane segments: 20365

Tortuosity Analysis:
Number of analyzed segments: 170
Mean tortuosity: 1.388

Coverage Analysis:
Membrane area: 260144 pixels
Coverage percentage: 24.81%
Membrane density: 248.09 per 1000 pixels

Complexity Analysis:
Number of branch points: 5710
Number of endpoints: 3703
Average segment length: 12.20 pixels
Branching complexity: 0.1263
Saved metrics to /content/drive/MyDrive/knowledge/

<ipython-input-3-70a6bb79fe38>:687: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sig_results['comparison'] = sig_results.apply(
<ipython-input-3-70a6bb79fe38>:689: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sig_results['difference'] = sig_results['mean2'] - sig_results['mean1']
<ipython-input-3-70a6bb79fe38>:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

Saved visualization of significant differences to /content/drive/MyDrive/knowledge/University/Master/Thesis/Analysis/flow3-x20/Overall/Membrane/Statistical_Analysis/significant_differences.png
Analysis complete. Results saved to /content/drive/MyDrive/knowledge/University/Master/Thesis/Analysis/flow3-x20/Overall/Membrane


<Figure size 1500x1000 with 0 Axes>